# Qu: demo notebook

## Set up and import

In [1]:
# Import relevant modules
from pathlib import Path

import torch

from qu.data import DataModel
from qu.ml import UNet2DLearner
from qu.ml import UNet2DSettings
from qu.demo import get_demo_segmentation_dataset

## Prepare the data and the model

In [2]:
# Get the demo dataset
data_root_path = get_demo_segmentation_dataset()
print(data_root_path)

# Set the working dir
working_dir = data_root_path

/home/aaron/.qu/data/demo_segmentation


In [3]:
# Notice: you can point tensorboard to {data_root_path}/runs.

In [4]:
# Scan for data
dataModel = DataModel()
dataModel.root_data_path = data_root_path
dataModel.scan()

True

In [5]:
# Get the actual split
train_image_names, train_mask_names, \
val_image_names, val_mask_names, \
test_image_names, test_mask_names = dataModel.training_split()

In [6]:
# Real split
print(f"Split: {len(train_image_names)}/{len(val_image_names)}/{len(test_image_names)}")

Split: 67/18/5


## Set up the Learner

In [7]:
# Check
print(f"Number of classes found in mask data: {dataModel.num_classes}")
print(f"Mask type estimated from mask data: {dataModel.mask_type}")

Number of classes found in mask data: 3
Mask type estimated from mask data: MaskType.TIFF_LABELS


In [8]:
# Initialize the settings
settings = UNet2DSettings()

In [9]:
# Change some settings
settings.roi_size = (512, 512)
settings.batch_sizes = (8, 1, 1, 1)
settings.num_epochs = 400

In [10]:
# Get the settings as dictionary
settings_dict = settings.to_dict()

In [11]:
# Initialize a learner
learner = UNet2DLearner(
    in_channels=1,
    out_channels=dataModel.num_classes,
    **settings_dict,
    working_dir=working_dir
)

In [12]:
# Add the data
learner.set_training_data(
    train_image_names,
    train_mask_names,
    val_image_names,
    val_mask_names,
    test_image_names,
    test_mask_names
)

## Train the model

In [13]:
# Free GPU memory
torch.cuda.empty_cache()

In [14]:
# Train the model
result = learner.train()

Using device 'cuda'.
--------------------------------------------------------------------------------
Epoch 1/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.9573
Batch 2/9: train_loss = 0.9549
Batch 3/9: train_loss = 0.9664
Batch 4/9: train_loss = 0.9668
Batch 5/9: train_loss = 0.9510
Batch 6/9: train_loss = 0.9372
Batch 7/9: train_loss = 0.9483
Batch 8/9: train_loss = 0.9433
Batch 9/9: train_loss = 0.9474
Average loss = 0.9525
--------------------------------------------------------------------------------
Epoch 2/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.9334
Batch 2/9: train_loss = 0.9433
Batch 3/9: train_loss = 0.9532
Batch 4/9: train_loss = 0.9597
Batch 5/9: train_loss = 0.9342
Batch 6/9: train_loss = 0.9255
Batch 7/9: train_loss = 0.9161
Batch 8/9: train_loss = 0.9135
Batch 9/9: train_loss = 0.9264
Average loss = 0.9339
-----------------------------

Batch 1/9: train_loss = 0.6449
Batch 2/9: train_loss = 0.6859
Batch 3/9: train_loss = 0.7215
Batch 4/9: train_loss = 0.7700
Batch 5/9: train_loss = 0.7049
Batch 6/9: train_loss = 0.6462
Batch 7/9: train_loss = 0.6748
Batch 8/9: train_loss = 0.7035
Batch 9/9: train_loss = 0.6827
Average loss = 0.6927
--------------------------------------------------------------------------------
Epoch 14/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.6473
Batch 2/9: train_loss = 0.6236
Batch 3/9: train_loss = 0.7201
Batch 4/9: train_loss = 0.7327
Batch 5/9: train_loss = 0.6747
Batch 6/9: train_loss = 0.6247
Batch 7/9: train_loss = 0.6352
Batch 8/9: train_loss = 0.6443
Batch 9/9: train_loss = 0.6757
Average loss = 0.6643
--------------------------------------------------------------------------------
Validation
--------------------------------------------------------------------------------
Global metric = 0.5250 
Class 'Background' metric 

Batch 6/9: train_loss = 0.3318
Batch 7/9: train_loss = 0.3266
Batch 8/9: train_loss = 0.3434
Batch 9/9: train_loss = 0.3554
Average loss = 0.3508
--------------------------------------------------------------------------------
Epoch 26/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.2949
Batch 2/9: train_loss = 0.3169
Batch 3/9: train_loss = 0.3534
Batch 4/9: train_loss = 0.3453
Batch 5/9: train_loss = 0.3148
Batch 6/9: train_loss = 0.3238
Batch 7/9: train_loss = 0.3146
Batch 8/9: train_loss = 0.2964
Batch 9/9: train_loss = 0.3008
Average loss = 0.3179
--------------------------------------------------------------------------------
Validation
--------------------------------------------------------------------------------
Global metric = 0.8791 
Class 'Background' metric = 0.9927 
Class 'Object' metric = 0.8549 
Class 'Border' metric = 0.7896 
New best global metric = 0.8791 at epoch: 26
Saved best model 'best_model_2021012

Batch 1/9: train_loss = 0.1829
Batch 2/9: train_loss = 0.1895
Batch 3/9: train_loss = 0.1901
Batch 4/9: train_loss = 0.1986
Batch 5/9: train_loss = 0.1796
Batch 6/9: train_loss = 0.1830
Batch 7/9: train_loss = 0.1847
Batch 8/9: train_loss = 0.1806
Batch 9/9: train_loss = 0.1950
Average loss = 0.1871
--------------------------------------------------------------------------------
Validation
--------------------------------------------------------------------------------
Global metric = 0.9029 
Class 'Background' metric = 0.9945 
Class 'Object' metric = 0.8892 
Class 'Border' metric = 0.8251 
New best global metric = 0.9029 at epoch: 38
Saved best model 'best_model_20210120_103257.pth'
--------------------------------------------------------------------------------
Epoch 39/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.1755
Batch 2/9: train_loss = 0.1785
Batch 3/9: train_loss = 0.1895
Batch 4/9: train_loss = 0.2098
Batch 5/

Global metric = 0.9120 
Class 'Background' metric = 0.9957 
Class 'Object' metric = 0.9210 
Class 'Border' metric = 0.8195 
--------------------------------------------------------------------------------
Epoch 51/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.1593
Batch 2/9: train_loss = 0.1481
Batch 3/9: train_loss = 0.1564
Batch 4/9: train_loss = 0.1541
Batch 5/9: train_loss = 0.1559
Batch 6/9: train_loss = 0.1568
Batch 7/9: train_loss = 0.1513
Batch 8/9: train_loss = 0.1449
Batch 9/9: train_loss = 0.1681
Average loss = 0.1550
--------------------------------------------------------------------------------
Epoch 52/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.1468
Batch 2/9: train_loss = 0.1433
Batch 3/9: train_loss = 0.1621
Batch 4/9: train_loss = 0.1641
Batch 5/9: train_loss = 0.1522
Batch 6/9: train_loss = 0.1554
Batch 7/9: train_loss = 0.1495
Batch 8/

...

Batch 1/9: train_loss = 0.0962
Batch 2/9: train_loss = 0.0976
Batch 3/9: train_loss = 0.1072
Batch 4/9: train_loss = 0.1003
Batch 5/9: train_loss = 0.1008
Batch 6/9: train_loss = 0.0998
Batch 7/9: train_loss = 0.1002
Batch 8/9: train_loss = 0.0985
Batch 9/9: train_loss = 0.1038
Average loss = 0.1005
--------------------------------------------------------------------------------
Validation
--------------------------------------------------------------------------------
Global metric = 0.9426 
Class 'Background' metric = 0.9986 
Class 'Object' metric = 0.9755 
Class 'Border' metric = 0.8537 
--------------------------------------------------------------------------------
Epoch 339/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.1092
Batch 2/9: train_loss = 0.0994
Batch 3/9: train_loss = 0.1029
Batch 4/9: train_loss = 0.1085
Batch 5/9: train_loss = 0.0927
Batch 6/9: train_loss = 0.1045
Batch 7/9: train_loss = 0.0963
Batch 8/9

Batch 1/9: train_loss = 0.0972
Batch 2/9: train_loss = 0.0852
Batch 3/9: train_loss = 0.1040
Batch 4/9: train_loss = 0.0960
Batch 5/9: train_loss = 0.0918
Batch 6/9: train_loss = 0.0921
Batch 7/9: train_loss = 0.0955
Batch 8/9: train_loss = 0.0918
Batch 9/9: train_loss = 0.0983
Average loss = 0.0946
--------------------------------------------------------------------------------
Epoch 352/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.0971
Batch 2/9: train_loss = 0.0954
Batch 3/9: train_loss = 0.0960
Batch 4/9: train_loss = 0.0902
Batch 5/9: train_loss = 0.0933
Batch 6/9: train_loss = 0.1058
Batch 7/9: train_loss = 0.1009
Batch 8/9: train_loss = 0.1042
Batch 9/9: train_loss = 0.0883
Average loss = 0.0968
--------------------------------------------------------------------------------
Validation
--------------------------------------------------------------------------------
Global metric = 0.9428 
Class 'Background' metric

Batch 8/9: train_loss = 0.0915
Batch 9/9: train_loss = 0.1012
Average loss = 0.0992
--------------------------------------------------------------------------------
Validation
--------------------------------------------------------------------------------
Global metric = 0.9410 
Class 'Background' metric = 0.9984 
Class 'Object' metric = 0.9736 
Class 'Border' metric = 0.8510 
--------------------------------------------------------------------------------
Epoch 365/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.0945
Batch 2/9: train_loss = 0.0874
Batch 3/9: train_loss = 0.1016
Batch 4/9: train_loss = 0.1006
Batch 5/9: train_loss = 0.0954
Batch 6/9: train_loss = 0.0910
Batch 7/9: train_loss = 0.0991
Batch 8/9: train_loss = 0.0946
Batch 9/9: train_loss = 0.1023
Average loss = 0.0963
--------------------------------------------------------------------------------
Epoch 366/400
-----------------------------------------------

Batch 6/9: train_loss = 0.0952
Batch 7/9: train_loss = 0.0997
Batch 8/9: train_loss = 0.1002
Batch 9/9: train_loss = 0.0931
Average loss = 0.0951
--------------------------------------------------------------------------------
Epoch 378/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.0924
Batch 2/9: train_loss = 0.0939
Batch 3/9: train_loss = 0.0872
Batch 4/9: train_loss = 0.1000
Batch 5/9: train_loss = 0.0997
Batch 6/9: train_loss = 0.1008
Batch 7/9: train_loss = 0.0957
Batch 8/9: train_loss = 0.0960
Batch 9/9: train_loss = 0.0936
Average loss = 0.0955
--------------------------------------------------------------------------------
Validation
--------------------------------------------------------------------------------
Global metric = 0.9440 
Class 'Background' metric = 0.9986 
Class 'Object' metric = 0.9760 
Class 'Border' metric = 0.8574 
--------------------------------------------------------------------------------

Global metric = 0.9423 
Class 'Background' metric = 0.9986 
Class 'Object' metric = 0.9757 
Class 'Border' metric = 0.8527 
--------------------------------------------------------------------------------
Epoch 391/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.0956
Batch 2/9: train_loss = 0.0973
Batch 3/9: train_loss = 0.0915
Batch 4/9: train_loss = 0.1138
Batch 5/9: train_loss = 0.1080
Batch 6/9: train_loss = 0.0971
Batch 7/9: train_loss = 0.0977
Batch 8/9: train_loss = 0.0871
Batch 9/9: train_loss = 0.1029
Average loss = 0.0990
--------------------------------------------------------------------------------
Epoch 392/400
--------------------------------------------------------------------------------
Batch 1/9: train_loss = 0.1004
Batch 2/9: train_loss = 0.0827
Batch 3/9: train_loss = 0.0965
Batch 4/9: train_loss = 0.1103
Batch 5/9: train_loss = 0.0938
Batch 6/9: train_loss = 0.0914
Batch 7/9: train_loss = 0.0902
Batch 

In [15]:
if not result:
    print(learner.get_message())

## Test predict

In [16]:
# Run prediction on the test set
result = learner.test_predict()

--------------------------------------------------------------------------------
Test prediction
--------------------------------------------------------------------------------
Loaded best metric model /home/aaron/.qu/data/demo_segmentation/runs/best_model_20210120_103257.pth.
Saved /home/aaron/.qu/data/demo_segmentation/tests/pred_t077.tif
Saved /home/aaron/.qu/data/demo_segmentation/tests/pred_t042.tif
Saved /home/aaron/.qu/data/demo_segmentation/tests/pred_t022.tif
Saved /home/aaron/.qu/data/demo_segmentation/tests/pred_t006.tif
Saved /home/aaron/.qu/data/demo_segmentation/tests/pred_t061.tif
Test prediction completed.


In [17]:
if not result:
    print(learner.get_message())

## Predict on new data

In [18]:
# Get path of best model
best_model_path = learner.get_best_model_path()
print(best_model_path)

/home/aaron/.qu/data/demo_segmentation/runs/best_model_20210120_103257.pth


In [19]:
# Just for the sake of example, predict on all images. 
# In practice, you would use brand new images here.
input_folder = Path(data_root_path) / "images"
output_folder = Path(working_dir) / "preds"

result = learner.predict(input_folder, output_folder, best_model_path)

--------------------------------------------------------------------------------
Prediction
--------------------------------------------------------------------------------
Loaded best metric model /home/aaron/.qu/data/demo_segmentation/runs/best_model_20210120_103257.pth.
Saved /home/aaron/.qu/data/demo_segmentation/preds/pred_train_t000.tif
Saved /home/aaron/.qu/data/demo_segmentation/preds/pred_train_t001.tif
Saved /home/aaron/.qu/data/demo_segmentation/preds/pred_train_t002.tif
Saved /home/aaron/.qu/data/demo_segmentation/preds/pred_train_t003.tif
Saved /home/aaron/.qu/data/demo_segmentation/preds/pred_train_t004.tif
Saved /home/aaron/.qu/data/demo_segmentation/preds/pred_train_t005.tif
Saved /home/aaron/.qu/data/demo_segmentation/preds/pred_train_t006.tif
Saved /home/aaron/.qu/data/demo_segmentation/preds/pred_train_t007.tif
Saved /home/aaron/.qu/data/demo_segmentation/preds/pred_train_t008.tif
Saved /home/aaron/.qu/data/demo_segmentation/preds/pred_train_t009.tif
Saved /home/aaro

In [20]:
if not result:
    print(learner._message)